In [ ]:
%matplotlib inline

# 3D interpolation

Interpolation of a three-dimensional regular grid.

## Trivariate

The :py`trivariate <pyinterp.trivariate>`{.interpreted-text role="func"}
interpolation allows obtaining values at arbitrary points in a 3D space
of a function defined on a grid.

The distribution contains a 3D field `tcw.nc` that will be used in this
help. This file is located in the `tests/dataset` directory at the root
of the project.

This method performs a bilinear interpolation in 2D space by considering
the axes of longitude and latitude of the grid, then performs a linear
interpolation in the third dimension. Its interface is similar to the
:py`bivariate <pyinterp.bivariate>`{.interpreted-text role="func"} class
except for a third axis, which is handled by this object.

::: {.note}
::: {.title}
Note
:::

When using a time axis, care must be taken to use the same unit of
dates, between the axis defined and the dates supplied during
interpolation. The function
:py`pyinterp.TemporalAxis.safe_cast`{.interpreted-text role="meth"}
automates this task and will warn you if there is an inconsistency
during the date conversion.
:::


In [ ]:
import os
import pathlib
import cartopy.crs
import matplotlib
import matplotlib.pyplot
import numpy
import pyinterp
import pyinterp.backends.xarray
import xarray

DATASET = pathlib.Path(os.environ['DATASET'])
TCW = DATASET.joinpath("tcw.nc")

The first step is to load the data into memory and create the
interpolator object:


In [ ]:
ds = xarray.open_dataset(TCW)
interpolator = pyinterp.backends.xarray.Grid3D(ds.tcw)

We will build a new grid that will be used to build a new interpolated
grid.


In [ ]:
mx, my, mz = numpy.meshgrid(numpy.arange(-180, 180, 0.25) + 1 / 3.0,
                            numpy.arange(-80, 80, 0.25) + 1 / 3.0,
                            numpy.array(["2002-07-02T15:00:00"],
                                        dtype="datetime64"),
                            indexing='ij')

We interpolate our grid using a :py`classical
<pyinterp.backends.xarray.Grid3D.trivariate>`{.interpreted-text
role="meth"}:


In [ ]:
trivariate = interpolator.trivariate(
    dict(longitude=mx.flatten(), latitude=my.flatten(), time=mz.flatten()))

# Bicubic on 3D grid

The grid used organizes the latitudes in descending order. We ask our
constructor to flip this axis in order to correctly evaluate the bicubic
interpolation from this 3D cube (only necessary to perform a bicubic
interpolation).


In [ ]:
interpolator = pyinterp.backends.xarray.Grid3D(ds.data_vars["tcw"],
                                               increasing_axes=True)

We interpolate our grid using a :py`bicubic
<pyinterp.backends.xarray.Grid3D.bicubic>`{.interpreted-text
role="meth"} interpolation in space followed by a linear interpolation
in the temporal axis:


In [ ]:
bicubic = interpolator.bicubic(
    dict(longitude=mx.flatten(), latitude=my.flatten(), time=mz.flatten()))

We transform our result cubes into a matrix.


In [ ]:
trivariate = trivariate.reshape(mx.shape).squeeze(axis=2)
bicubic = bicubic.reshape(mx.shape).squeeze(axis=2)
lons = mx[:, 0].squeeze()
lats = my[0, :].squeeze()

Let\'s visualize our results.


In [ ]:
fig = matplotlib.pyplot.figure(figsize=(5, 8))
fig.patch.set_alpha(0.0)
ax1 = fig.add_subplot(
    211, projection=cartopy.crs.PlateCarree(central_longitude=180))
pcm = ax1.pcolormesh(lons,
                     lats,
                     trivariate.T,
                     cmap='jet',
                     transform=cartopy.crs.PlateCarree(),
                     vmin=0,
                     vmax=80)
ax1.coastlines()
ax1.set_extent([80, 170, -45, 30], crs=cartopy.crs.PlateCarree())
ax1.set_title("Trilinear")

ax2 = fig.add_subplot(
    212, projection=cartopy.crs.PlateCarree(central_longitude=180))
pcm = ax2.pcolormesh(lons,
                     lats,
                     bicubic.T,
                     cmap='jet',
                     transform=cartopy.crs.PlateCarree(),
                     vmin=0,
                     vmax=80)
ax2.coastlines()
ax2.set_extent([80, 170, -45, 30], crs=cartopy.crs.PlateCarree())
ax2.set_title("Spline & Linear in time")
fig.colorbar(pcm, ax=[ax1, ax2], shrink=0.8)
fig.show()